In [5]:
import os
from openai import OpenAI

In [14]:
client = OpenAI(base_url="https://api.proxyapi.ru/openai/v1",
                api_key=os.environ['KEY'])

In [ ]:
resp = client.chat.completions.create(
    model='gpt-4o-mini',
    max_completion_tokens=5,
    messages=[{'role': 'user', 'content': 'как приготовить лосося?'}]
)
resp.choices[0].messege.content